# The Pale Blue Dot Challenge Workflow <br>
By:<br>

**Julius Kimani** <br>
**Gabriel Paul** <br>
**Leone Majur** <br>

The workflow comprised the following: <br>

* Using Google Earth Engine's Python API
* Defining an AOI (area of interest) in the Mt. Kenya region
* Importing multiple Landsat images from the dates before and after Fire ocurrence
* Determining which images are best for analysis
* Creating various NDVI map outputs to assess the amount of fire damage

## Python libraries in this workflow
* [pandas](https://pandas.pydata.org/docs/getting_started/index.html)<br>
* [geopandas](https://geopandas.org/en/stable/gallery/index.html)<br>
* [google earth engine](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard) <br>
* [matplotlib](https://matplotlib.org/) <br>
* [folium](https://realpython.com/python-folium-web-maps-from-data/) <br>


## Importing Libraries

In [2]:
# the regulars
import geopandas as gpd
import matplotlib.pyplot as plt

# work with dates
from datetime import datetime as dt

# allow images to display
from IPython.display import Image

# earth engine
import ee

## Authenticate Earth Engine

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AfJohXnF9dJaYtx17E2iZM5CQCzXmQF1sxLDkBNHgFKLc-U_KYiySHoQehE



Successfully saved authorization token.


For this workflow, we use Google Earth Engine's "USGS Landsat 8 Level 2, Collection 2, Tier 1" <br>

* [EE Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)


  ## Defining Filters

In [6]:
# coordinates of the bobcat fire
lat = -0.18
lon = 37.3

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2018-12-09'

# end date
end_date = '2019-04-09'

## Getting Landsat 8 data

In [7]:
# get the satellite data
# ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
# landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [9]:
# how many images did we get?
print('Total number:', landsat.size().getInfo())

Total number: 8


In [10]:
# information about the first image in our collection
landsat.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 209685, 0, -30, 115815]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 209685, 0, -30, 115815]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 209685, 0, -30, 115815]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 209685, 0, -30, 115815]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',


In [14]:
# what about cloud cover of our first image?
landsat.first().get('CLOUD_COVER').getInfo()

28.2

In [15]:
# when was this image taken?
date = ee.Date(landsat.first().get('system:time_start')).getInfo()['value']
time = date/1000
date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
date

'2018-12-15 07:42:39'

## Bands

[Source: USGS](https://www.usgs.gov/media/images/landsat-8-band-designations)

![title](img/bands.jpg)



In [16]:
# what bands did we get?
landsat.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [17]:
# put the images in a list
landsat_list = landsat.toList(landsat.size());

## Displaying the Satellite Images <br>

* [what are the min/max values?](https://gis.stackexchange.com/questions/304180/what-are-the-min-and-max-values-of-map-addlayer-on-google-earth-engine)

In [20]:
# set some parameters for image
parameters = {
                'min': 0,
                'max': 2500,
                'dimensions': 600,
                'bands': ['B4', 'B3', 'B2']
             }

In [22]:
# loop through each image and display it
for i in range(landsat.size().getInfo()):

    # when was this image taken?
    date = ee.Date(ee.Image(landsat_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
    
    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 0 2018-12-15 07:42:39 Cloud cover: 28.2


Image # 1 2018-12-31 07:42:39 Cloud cover: 43.92


Image # 2 2019-01-16 07:42:37 Cloud cover: 33.61


Image # 3 2019-02-01 07:42:33 Cloud cover: 10.02


Image # 4 2019-02-17 07:42:31 Cloud cover: 25.69


Image # 5 2019-03-05 07:42:26 Cloud cover: 27.79


Image # 6 2019-03-21 07:42:22 Cloud cover: 2.88


Image # 7 2019-04-06 07:42:18 Cloud cover: 17.22


## Selecting images, zooming in <br>

Now that we had inspected our collection of images, we picked and chose which ones were relevant for our study. Ideally, we picked images for before and after the fire to be able to assess the level of damage. <br>

We also created an ROI (region of interest) and zoomed in to the area of interest. We did so by applying a 25km buffer around our POI.

In [23]:
# create a list of images we want (before and after)
landsat_sequence = [3,6]

In [24]:
# Define a region of interest with a buffer zone of 25 km
roi = poi.buffer(25000) # meters

In [25]:
parameters = {'min': 0,
              'max': 2500,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region':roi
              }

In [36]:
for i in landsat_sequence:
    # when was this image taken?
    date = ee.Date(ee.Image(landsat_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 3 2019-02-01 07:42:33 Cloud cover: 10.02


Image # 6 2019-03-21 07:42:22 Cloud cover: 2.88


## Normalized Difference Vegetation Index (NDVI) <br>

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, often from a space platform, assessing whether or not the target being observed contains live green vegetation.

![title](img/ndvi.png)

* [Source:Agricolus](https://www.agricolus.com/en/vegetation-indices-ndvi-ndmi/) <br>

### Calculating NDVI in Google Earth Engine <br>

* https://developers.google.com/earth-engine/tutorials/tutorial_api_06

In [27]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [28]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Date(ee.Image(landsat_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
    
    # print some information
    print('Image #',i,date)
    
    # display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['B5', 'B4']).getThumbUrl(ndvi_parameters)))

Image # 3 2019-02-01 07:42:33


Image # 6 2019-03-21 07:42:22


## Folium <br>

We couldn't end the workflow without an interactive map. For this, we used folium, which uses the open-source javascript mapping library leaflet. <br>

* https://python-visualization.github.io/folium/quickstart.html <br>
  
    Google earth engine works with folium: <br>

* https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard?hl=en#interactive_mapping_using_folium

In [29]:
# A simple folium map
import folium
m = folium.Map(location=[lat,lon])
m

In [30]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [31]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Date(ee.Image(landsat_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['B5', 'B4']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

## Saving the folium map as an html file

In [32]:
my_map.save('MtKE.html')

## Resources <br>


* [Earth Lab](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/) <br>
* [Google's Python Tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)